# Checking FYs for JEFF-4.0

In [ ]:
import sandy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("FY_JEFF4_somenuclides.csv", index_col=0)
dferr = pd.read_csv("DFY_JEFF4_somenuclides.csv", index_col=0)

In [ ]:
nuclides = [380900, 551370, 601430, 601450, 601440,  601450,  601460,  601470,  601480,  601500]

In [ ]:
fy = sandy.get_endf6_file("jeff_33", "nfpy", 922350)
d = sandy.Fy.from_endf6(fy).data.query("MT==459 & E==2.53e-2")
d_33 = d[d.ZAP.isin(nuclides)].assign(LIB="JEFF-3.3")

In [ ]:
fy = sandy.get_endf6_file("jeff_311", "nfpy", 922350)
d = sandy.Fy.from_endf6(fy).data.query("MT==459 & E==2.53e-2")
d_311 = d[d.ZAP.isin(nuclides)].assign(LIB="JEFF-3.1.1")

In [ ]:
data = pd.concat([
    d_311,
    d_33,
], ignore_index=True)

In [ ]:
vals = data.query("ZAM==922350").pivot_table(index="LIB", columns="ZAP", values="FY")
vals.columns = pd.Index(map(sandy.zam.zam2nuclide, vals.columns), name="ZAP")
vals = pd.concat([df, vals])
vals

In [ ]:
err = data.query("ZAM==922350").pivot_table(index="LIB", columns="ZAP", values="DFY")
err.columns = pd.Index(map(sandy.zam.zam2nuclide, err.columns), name="ZAP")
err = pd.concat([dferr, err])
err

In [ ]:
%%capture --no-display
fig, ax = plt.subplots(figsize=(10, 4.8), dpi=300)

vals2 = (vals/vals.iloc[-1] - 1)*100 
err2 = err/vals.iloc[-1]*100
err2 = err/vals*100
step = 1 / (vals2.T.columns.size + 1)
for i, LIB in enumerate(vals2.T.columns):
    x = np.arange(vals2.T[LIB].shape[0])
    ax.bar(data=vals2.T[LIB].reset_index(), x=x+i*step, height=LIB, yerr=err2.T[LIB], capsize=2, width=step)
ax.legend(labels=vals2.T.columns)
ticks = ax.get_xticks()
ax.set_xticklabels(vals2.columns.values)
ax.set_xticks(np.arange(len(vals2.columns)) + step)
ax.set(
    ylabel="CFY deviation from JEFF-3.3 $\\pm 1\\sigma$",
)
fig.tight_layout()

In [ ]:
%%capture --no-display
fig, ax = plt.subplots(figsize=(10, 4.8), dpi=100)

vals2 = (vals/vals.iloc[-2] - 1)*100 
err2 = err/vals*100
step = 1 / (vals2.T.columns.size + 1)
for i, LIB in enumerate(vals2.T.columns):
    x = np.arange(vals2.T[LIB].shape[0])
    ax.bar(data=vals2.T[LIB].reset_index(), x=x+i*step, height=LIB, yerr=err2.T[LIB], capsize=2, width=step)
ax.legend(labels=vals2.T.columns)
ticks = ax.get_xticks()
ax.set_xticklabels(vals2.columns.values)
ax.set_xticks(np.arange(len(vals2.columns)) + step)
ax.set(
    ylabel="CFY deviation from JEFF-3.1.1 $\\pm 1\\sigma$",
)
fig.tight_layout();

In [ ]:
%%capture --no-display
fig, axs = plt.subplots(1, 3, figsize=(10, 4.8), dpi=300, gridspec_kw={'width_ratios': [4, 1, 1]})

ax = axs[0]
vals2 = vals.iloc[:, :4]
err2 = err.iloc[:, :4]
step = 1 / (vals2.T.columns.size + 1)
for i, LIB in enumerate(vals2.T.columns):
    x = np.arange(vals2.T[LIB].shape[0])
    ax.bar(data=vals2.T[LIB].reset_index(), x=x+i*step, height=LIB, yerr=err2.T[LIB], capsize=2, width=step)
ax.legend(labels=vals2.T.columns, ncol=2)
ticks = ax.get_xticks()
ax.set_xticklabels(vals2.columns.values)
ax.set_xticks(np.arange(len(vals2.columns)) + step)
ax.set(
    ylabel="cumulative fission yield $\\pm 1\\sigma$",
    ylim=(.05, None),
)

ax = axs[1]
vals2 = vals.iloc[:, 4:5]
err2 = err.iloc[:, 4:5]
step = 1 / (vals2.T.columns.size + 1)
for i, LIB in enumerate(vals2.T.columns):
    x = np.arange(vals2.T[LIB].shape[0])
    ax.bar(data=vals2.T[LIB].reset_index(), x=x+i*step, height=LIB, yerr=err2.T[LIB], capsize=2, width=step)
ax.legend(labels=vals2.T.columns)
ticks = ax.get_xticks()
ax.set_xticklabels(vals2.columns.values)
ax.set_xticks(np.arange(len(vals2.columns)) + step)
ax.get_legend().remove()
ax.set(
    ylabel=None,
    ylim=(.038, 0.042),
)

ax = axs[2]
vals2 = vals.iloc[:, 7:8]
err2 = err.iloc[:, 7:8]
step = 1 / (vals2.T.columns.size + 1)
for i, LIB in enumerate(vals2.T.columns):
    x = np.arange(vals2.T[LIB].shape[0])
    ax.bar(data=vals2.T[LIB].reset_index(), x=x+i*step, height=LIB, yerr=err2.T[LIB], capsize=2, width=step)
ax.legend(labels=vals2.T.columns)
ticks = ax.get_xticks()
ax.set_xticklabels(vals2.columns.values)
ax.set_xticks(np.arange(len(vals2.columns)) + step)
ax.get_legend().remove()
ax.set(
    ylabel=None,
    ylim=(.016, 0.0175),
)

fig.tight_layout();